#  Track Analysis

In order to properly analyze our GPX tracks, we need to put all of them into the same _frame-of-reference_.   This is accomplished by the following algorithm...

* Create a set of variables with the initialized values
    * Current Coordinate = Starting Coordinate
        * $P_{\textrm{cur}} = P_{\textrm{start}}$
        * **NOTE:** Starting coordinate $P_{\textrm{start}}$ is defined below as a fixed variable
    * Distance Threshold = Small distance in meters
        * $d_{\textrm{thresh}} = 20 meters$
        * This is the distance between a possible point and the current point for consideration.
    * Step Distance = Small distance in meters.
        * $d_{\textrm{step}} = 20 meters$
        * This is the distance that the route will jump using the average angle
    * Waypoint list = empty array
        * $\hat{P}_{\textrm{waypoints}} = []$

## Step 0: Globals

In [ ]:
database_path = 'bike_data.db'

start_coord = (39.5989743, -104.8609468)
end_coord   = (39.75428108249532, -105.00085402872664)

dist_thresh_m = 100
step_dist_m = 25

epsg_code = 32613

## Step 1: Import Required Libraries

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from ipyleaflet import Map, Marker, Polygon, Polyline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import math, cmath
import re
from geographiclib.geodesic import Geodesic
from pyproj import CRS, Proj, Transformer

Setup the database.

In [3]:
# SQLAlchemy connectable 
conn = create_engine( 'sqlite:///' + database_path ).connect()

#  For each segment, we need to create a track for each dataset
dataset_ids = pd.read_sql_query('SELECT DISTINCT datasetId FROM point_list', conn)

Setup Proj

In [4]:
#  Setup the Projection Transformer
crs = CRS.from_epsg( epsg_code )
proj = Transformer.from_crs(crs.geodetic_crs, crs)
utm_zone = int(re.findall("\d+", crs.utm_zone)[0])
print('UTM Zone: {}'.format(utm_zone))

UTM Zone: 13


## Step 2: Import points 

In [5]:
#  For each dataset, load the points w.r.t. each dataset-id, sorted by time.
points_by_dataset = {}
for dataset_id in dataset_ids['datasetId']:
        
    #  Create a full track of the segment
    sql_query = 'SELECT * FROM point_list WHERE datasetId = {} ORDER BY timestamp'.format( dataset_id )
    points_by_dataset[dataset_id] = { 'points': pd.read_sql_query( sql_query, conn ) }

## Step 3: Compute Initial Starting Point

In [6]:
def mean_angle(angles):
    return math.degrees(cmath.phase(sum(cmath.rect(1, math.radians(d)) for d in angles)/len(angles)))

def Cross( a, b ):
    return a[0] * b[1] - a[1] * b[0]
    
def Is_In_Front( anchor_point, test_point, angle ):
    
    #  Create a point to the side of the anchor to create a line we can compare
    side_point = ( anchor_point[0] + math.cos( math.radians( angle ) + math.pi/2.0 ),
                   anchor_point[1] + math.sin( math.radians( angle ) + math.pi/2.0 ) )
    
    p1 = ( test_point[0] - side_point[0],
           test_point[1] - side_point[1] )
    p2 = ( test_point[0] - anchor_point[0],
           test_point[1] - anchor_point[1] )
    res = Cross( p1, p2 )
    print('Result: {}'.format(res))
    if res >= 0:
        return True
    return False

def Distance_L2( p1, p2 ):
    return math.sqrt( math.pow( p1[0] * p2[0], 2) + math.pow( p1[1] * p2[1], 2) )

waypoint_list = []
current_point = start_coord
matching_points = []

current_dataset_idx = {}
for dataset_id in dataset_ids['datasetId']:
    current_dataset_idx[dataset_id] = 0

for i in range(0,100):
    print('Start of Loop: {}'.format(i))
    #  Look for the nearest first points within the threshold
    temp_angles = []
    for dataset_id in dataset_ids['datasetId']:
        print(dataset_id)
        test_point = ( points_by_dataset[dataset_id]['points']['latitude'][current_dataset_idx[dataset_id]],
                       points_by_dataset[dataset_id]['points']['longitude'][current_dataset_idx[dataset_id]] )
    
        geod = Geodesic.WGS84.Inverse( start_coord[0], start_coord[1], test_point[0], test_point[1] )
    
        if geod['s12'] < dist_thresh_m:
        
            #  Compute angle to new point
            temp_angles.append( geod['azi1'])
            matching_points.append( (dataset_id, current_dataset_idx[dataset_id]) )
        else:
            pass
        
    #  If no matching points, break for now
    if len(temp_angles) < 1:
        print('No matching points. halting.')
        break

    avg_angle = mean_angle( temp_angles )
    print('avg-angle: {}'.format(avg_angle))

    #  Compute new point
    geod = Geodesic.WGS84.Direct( current_point[0],
                                  current_point[1],
                                  avg_angle,
                                  step_dist_m )


    print('Converting Coordinate')
    (easting,northing) = proj.transform( geod['lat2'], geod['lon2'] )
    current_point = ( geod['lat2'], geod['lon2'], utm_zone, easting, northing )
    waypoint_list.append( current_point )

    #  Update Current Indices
    print('Updating Indices')
    for dataset_id in dataset_ids['datasetId']:
        print('dataset: {}'.format(dataset_id))
        point = ( points_by_dataset[dataset_id]['points']['easting'][current_dataset_idx[dataset_id]],
                  points_by_dataset[dataset_id]['points']['northing'][current_dataset_idx[dataset_id]] )
        print(point)
        
        # Update the indeces until the next coordinate is no longer in "front"
        while Is_In_Front( current_point, point, avg_angle ) and ( Distance_L2( point, current_point ) < dist_thresh_m ):
            print('Is-In-Front: {}'.format(res))
            if not res:
                break
            print('Current IDX: {}'.format(current_dataset_idx[dataset_id]))
            current_dataset_idx[dataset_id] += 1
            point = ( points_by_dataset[dataset_id]['points']['easting'][current_dataset_idx[dataset_id]],
                      points_by_dataset[dataset_id]['points']['northing'][current_dataset_idx[dataset_id]] )

    print('End of Loop')
    
print('Waypoints:\n{}'.format(waypoint_list))
    

Start of Loop: 0
0
1
2
avg-angle: 63.33839655899835
Converting Coordinate
Updating Indices
dataset: 0
(511958.8646676866, 4383300.1266438095)
Result: 4147036.1240234375
dataset: 1
(511946.31110278185, 4383256.601921191)
Result: 4146991.5942382812
dataset: 2
(511278.40225939546, 4386385.763946345)
Result: 4149488.3354492188
End of Loop
Start of Loop: 1
0
1
2
avg-angle: 63.33839655899835
Converting Coordinate
Updating Indices
dataset: 0
(511958.8646676866, 4383300.1266438095)
Result: 4147036.1235351562
dataset: 1
(511946.31110278185, 4383256.601921191)
Result: 4146991.5942382812
dataset: 2
(511278.40225939546, 4386385.763946345)
Result: 4149488.3349609375
End of Loop
Start of Loop: 2
0
1
2
avg-angle: 63.33839655899835
Converting Coordinate
Updating Indices
dataset: 0
(511958.8646676866, 4383300.1266438095)
Result: 4147036.123046875
dataset: 1
(511946.31110278185, 4383256.601921191)
Result: 4146991.59375
dataset: 2
(511278.40225939546, 4386385.763946345)
Result: 4149488.3349609375
End of 

## Visualize Results

In [7]:
centroid_pt = [0, 0]

#  Create the polyline and list of points
marker_list = []
polyline   = []

for point in waypoint_list:
    marker_list.append( [point[0],point[1]] )
    centroid_pt[0] += point[0]
    centroid_pt[1] += point[1]
centroid_pt[0] /= len( waypoint_list )
centroid_pt[1] /= len( waypoint_list )

#  Build Map Visualization    
sector_map = Map( center=centroid_pt, zoom=14 )

#  Add Route Markers
for marker in marker_list:
    marker = Marker( location=marker, draggable=False )
    sector_map.add_layer( marker )
   
#  Create the Polyline
route_poly = Polyline( locations= marker_list,
                       color='blue',
                       fill=False )
sector_map.add_layer( route_poly )
    
#sector_map.layout.height="400px"
sector_map

Map(center=[39.604076731164156, -104.84781023037235], controls=(ZoomControl(options=['position', 'zoom_in_text…